## 1. The Multinomial Logit (MNL) Model:

**The Problem**: For each individual (or observation), we have a set of alternative choices. The goal is to predict the probability of each choice based on some features (independent variables).

### Mathematical Representation:

**Utility**: The utility $ U_{ij} $ that individual $ i $ derives from alternative $ j $ can be represented as:

$$ U_{ij} = V_{ij} + \varepsilon_{ij} $$

Where:
- $ V_{ij} $ is the systematic component of the utility, which is observed.
- $ \varepsilon_{ij} $ is the random component of the utility, which is unobserved.

The systematic component is often represented as a linear function of the predictors:

$$ V_{ij} = \beta X_{ij} $$

Where:
- $ \beta $ are the parameters to be estimated.
- $ X_{ij} $ are the predictors for individual $ i $ and alternative $ j $.

### The MNL Model:

For the MNL model, the probability $ P_{ij} $ that individual $ i $ chooses alternative $ j $ is:

$$ P_{ij} = \frac{e^{V_{ij}}}{\sum_{k} e^{V_{ik}}} $$

Where:
- $ k $ ranges over all available alternatives for individual $ i $.

## 2. Estimation:

### Maximum Likelihood Estimation:

To estimate the parameters $ \beta $, we maximize the likelihood function. For an individual $ i $, who chose alternative $ j $, the likelihood contribution is $ P_{ij} $. Hence, the log-likelihood $ LL $ for the sample is:

$$ LL(\beta) = \sum_{i} \log(P_{ij}) $$

Where:
- The sum is over all individuals.
- $ j $ is the alternative chosen by individual $ i $.

The estimated $ \beta $ will be those values that maximize this $ LL(\beta) $.

## 3. Dataset Structure:

### Long Format:

When building an MNL model, datasets are often transformed into "long" format. This means each row represents an individual-alternative combination.

For our example:
- Each person's characteristics (like income) get repeated for each available alternative (work, shopping, leisure).
- An additional binary variable indicates if the alternative was the chosen one.

## 4. Why Long Format?

It's beneficial for the following reasons:
- Easy to set up the systematic utility calculation: For each row, you just multiply the attributes by the parameters and sum them up.
- Straightforward computation of choice probabilities: For each individual, compute $ e^{V_{ij}} $ for each alternative $ j $, sum them up, and then divide $ e^{V_{ij}} $ by this sum.

## 5. PyLogit Implementation:

When you translate this to PyLogit:
- You specify which variables in your dataset correspond to the predictors in your utility function.
- PyLogit takes care of setting up the systematic utility function and maximizing the log-likelihood.

## Conclusion:

The Multinomial Logit model is an elegant way of predicting choices among multiple alternatives. It assumes the unobserved parts of utilities are independently and identically Gumbel distributed, leading to the convenient formula for the choice probabilities. The choice of predictors and their functional forms in the systematic utility part are critical, and often driven by theory, data exploration, and model testing.


### Step 1: Modified Sample Dataset Creation

Let's assume each person makes between 1 and 3 trips a day:

1.  `person_id` (unique identifier for each person)
2.  `trip_id` (unique identifier for each trip)
3.  `income` (annual income in $1000s)
4.  `num_vehicles` (number of vehicles in the household)
5.  `activity_choice` (1 for "work", 2 for "shopping", 3 for "leisure")

In [19]:
import pandas as pd

data = {
    "person_id": [1, 1, 1, 2, 3, 3, 4],
    "trip_id": [1, 2, 3, 1, 1, 2, 1],
    "age": [25, 25, 25, 45, 35, 35, 60],
    "gender": ['M', 'M', 'M', 'F', 'F', 'F', 'M'],
    "household_type": ['Joint', 'Joint', 'Joint', 'Single', 'Single', 'Single', 'Single'],
    "house_type": ['Apartment', 'Apartment', 'Apartment', 'House', 'Apartment', 'Apartment', 'House'],
    "number_adults": [2, 2, 2, 1, 1, 1, 1],
    "num_children": [1, 1, 1, 0, 0, 0, 0],
    "num_vehicles": [2, 2, 2, 1, 1, 1, 0],
    "activity_choice": [1, 2, 3, 3, 2, 1, 3]
}

df = pd.DataFrame(data)
df.head()

,person_id,trip_id,age,gender,household_type,house_type,number_adults,num_children,num_vehicles,activity_choice
0,1,1,25,M,Joint,Apartment,2,1,2,1
1,1,2,25,M,Joint,Apartment,2,1,2,2
2,1,3,25,M,Joint,Apartment,2,1,2,3
3,2,1,45,F,Single,House,1,0,1,3
4,3,1,35,F,Single,Apartment,1,0,1,2


In [28]:
# Dictionary of possible choices
possible_choices = {
    "work": 1,
    "shopping": 2,
    "leisure": 3
}

# Create a base dataframe that will be replicated for each choice
num_choices = len(possible_choices)
df_long = df.loc[df.index.repeat(num_choices)].reset_index(drop=True)

# Create a choice column 
df_long["choice"] = list(possible_choices.values()) * len(df)

# Create an indicator for the chosen activity
df_long["chosen"] = (df_long["choice"] == df_long["activity_choice"]).astype(int)

# Sorting the data so rows related to the same trip are contiguous
df_long = df_long.sort_values(by=["person_id", "trip_id"])

df_long['gender'] = df_long['gender'].astype('category').cat.codes
df_long['household_type'] = df_long['household_type'].astype('category').cat.codes
df_long['house_type'] = df_long['house_type'].astype('category').cat.codes

df_long = df_long.sort_values(by=["trip_id", "choice"])

df_long.head()

,person_id,trip_id,age,gender,household_type,house_type,number_adults,num_children,num_vehicles,activity_choice,choice,chosen
0,1,1,25,1,0,0,2,1,2,1,1,1
9,2,1,45,0,1,1,1,0,1,3,1,0
12,3,1,35,0,1,0,1,0,1,2,1,0
18,4,1,60,1,1,1,1,0,0,3,1,0
1,1,1,25,1,0,0,2,1,2,1,2,0


In [29]:
import pylogit as pl
from collections import OrderedDict

# Create a specification that maps the predictors to the alternatives
specification = OrderedDict([
    ("age", [1, 2, 3]),
    ("gender", [1, 2, 3]),
    ("household_type", [1, 2, 3]),
    ("house_type", [1, 2, 3]),
    ("number_adults", [1, 2, 3]),
    ("num_children", [1, 2, 3]),
    ("num_vehicles", [1, 2, 3])
])

# Create meaningful names for each coefficient in the model output
names = OrderedDict([
    ("age", ["Age Coef for Work", "Age Coef for Shopping", "Age Coef for Leisure"]),
    ("gender", ["Gender Coef for Work", "Gender Coef for Shopping", "Gender Coef for Leisure"]),
    ("household_type", ["Household Type Coef for Work", "Household Type Coef for Shopping", "Household Type Coef for Leisure"]),
    ("house_type", ["House Type Coef for Work", "House Type Coef for Shopping", "House Type Coef for Leisure"]),
    ("number_adults", ["Number of Adults Coef for Work", "Number of Adults Coef for Shopping", "Number of Adults Coef for Leisure"]),
    ("num_children", ["Number of Children Coef for Work", "Number of Children Coef for Shopping", "Number of Children Coef for Leisure"]),
    ("num_vehicles", ["Number of Vehicles Coef for Work", "Number of Vehicles Coef for Shopping", "Number of Vehicles Coef for Leisure"])
])

# Create the choice model
model = pl.create_choice_model(data=df_long, 
                               alt_id_col="choice", 
                               obs_id_col="trip_id", 
                               choice_col="chosen",
                               specification=specification,
                               model_type="MNL",  # Multinomial Logit
                               names=names)

# Estimate the model parameters
import numpy as np
model.fit_mle(np.zeros(21))  # 21 predictors in this example

# Print the estimation results
print(model.get_statsmodels_summary())


Log-likelihood at zero: -14.6218
Initial Log-likelihood: -14.6218


Estimation Time for Point Estimation: 0.05 seconds.
Final log-likelihood: -14.2591
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      chosen   No. Observations:                    3
Model:             Multinomial Logit Model   Df Residuals:                      -18
Method:                                MLE   Df Model:                           21
Date:                     Tue, 29 Aug 2023   Pseudo R-squ.:                   0.025
Time:                             11:29:21   Pseudo R-bar-squ.:              -1.411
AIC:                                70.518   Log-Likelihood:                -14.259
BIC:                                51.589   LL-Null:                       -14.622
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Age Coef for Work                  

C:\Users\ssanjay\AppData\Roaming\Python\Python38\site-packages\scipy\optimize\_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
c:\Users\ssanjay\AppData\Local\Continuum\anaconda3\envs\OSMnx\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1228: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.cov)),
c:\Users\ssanjay\AppData\Local\Continuum\anaconda3\envs\OSMnx\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),


In [36]:
model.fit_summary

Number of Parameters                                                     21
Number of Observations                                                    3
Null Log-Likelihood                                              -14.621758
Fitted Log-Likelihood                                            -14.259075
Rho-Squared                                                        0.024804
Rho-Bar-Squared                                                   -1.411412
Estimation Message        Desired error not necessarily achieved due to ...
dtype: object

### 1\. Predict Probabilities:

Using the `predict()` method, you can compute the probability of each alternative in your dataset.

In [138]:
import pandas as pd
import numpy as np
import pylogit as pl

# Assume you've already defined possible_choices and estimated a model named "model"

def generate_activity_sequence(person_attributes, model, max_activities=10):
    # Initialize the activity sequence with 'home' as the first activity
    sequence = [(1, "home")]
    
    # Trip ID will be used to uniquely identify each activity for the individual. Start with 2 because 'home' has ID 1.
    trip_id = 2
    
    # Create a base dataframe for the individual
    person_df = pd.DataFrame([person_attributes])
    
    # Loop until we generate a max number of activities
    for _ in range(max_activities - 1):  # Adjusted for initial 'home'
        # Replicate the row for each possible activity
        activity_df = person_df.loc[person_df.index.repeat(len(possible_choices))].reset_index(drop=True)
        activity_df["choice"] = list(possible_choices.values())
        activity_df["trip_id"] = trip_id
        
        # Predict probabilities for each activity
        predicted_probabilities = model.predict(activity_df)

        # Ensure the chosen activity is not the same as the previous activity
        last_activity = sequence[-1][1]
        chosen_activity = last_activity
        while chosen_activity == last_activity:
            # Simulate the choice based on probabilities
            random_draw = np.random.rand()
            chosen_activity_index = np.searchsorted(predicted_probabilities.cumsum(), random_draw)
            chosen_activity = list(possible_choices.keys())[chosen_activity_index]
        
        # Append chosen activity to the sequence
        sequence.append((trip_id, chosen_activity))
        
        # Increment the trip_id for the next trip
        trip_id += 1
        
        # Let's assume if the chosen activity is "home", then the day ends
        if chosen_activity == "home":
            break
            
        # Update conditions or states here if necessary 
        # E.g., if certain activities affect subsequent choices
        
    # Check if the last activity is not "home" and add it to the end if not
    if sequence[-1][1] != "home":
        sequence.append((trip_id, "home"))

    return sequence

# Define attributes for a new individual
new_individual = {
    "age": 35,
    "gender": 0,  # 0 for male, 1 for female
    "household_type": 1,
    "house_type": 2,
    "number_adults": 2,
    "num_children": 1,
    "num_vehicles": 1
}

# Generate activity sequence
activity_sequence = generate_activity_sequence(new_individual, model, max_activities=4)
print(activity_sequence)


[(1, 'home'), (2, 'leisure'), (3, 'shopping'), (4, 'leisure'), (5, 'home')]


## Attributes

In our model, we had several attributes, each affecting the choice of an activity. The attributes included:

- $ \text{age} $

- $ \text{gender} $

- $ \text{household\_type} $

- $ \text{house\_type} $

- $ \text{number\_adults} $

- $ \text{num\_children} $

- $ \text{num\_vehicles} $

Each attribute $ x_{k} $ is associated with a parameter $ \beta_{k} $. The deterministic part of the utility $ V_{nj} $ for each activity $ j $ for individual $ n $ can thus be represented as:

$$ V_{nj} = \sum_k \beta_{k} x_{knj} $$

Where $ x_{knj} $ represents the value of attribute $ k $ for individual $ n $ and activity $ j $.

## Worked Out Example:

Let's take a fictional individual, and walk through the calculation:

**Individual Attributes:**

- $ \text{age} = 25 $

- $ \text{gender} = \text{Male} $ (Assume male is represented as 1 and female as 0)

- $ \text{household\_type} = \text{Single} $ (Assume single is represented as 1, family as 2)

- $ \text{house\_type} = \text{Apartment} $ (Assume apartment is 1, house is 2)

- $ \text{number\_adults} = 1 $

- $ \text{num\_children} = 0 $

- $ \text{num\_vehicles} = 1 $

**Estimated Coefficients (from our model):**

- $ \beta_{\text{age}} = 0.05 $

- $ \beta_{\text{gender}} = 0.1 $

- $ \beta_{\text{household\_type}} = -0.2 $

- $ \beta_{\text{house\_type}} = 0.15 $

- $ \beta_{\text{number\_adults}} = 0.3 $

- $ \beta_{\text{num\_children}} = -0.4 $

- $ \beta_{\text{num\_vehicles}} = 0.25 $

For the sake of this example, let's compute the deterministic utility $ V_{n} $ for the activity "Shopping":

$$ V_{n\text{, Shopping}} = (0.05 \times 25) + (0.1 \times 1) + (-0.2 \times 1) + (0.15 \times 1) + (0.3 \times 1) - (0.4 \times 0) + (0.25 \times 1) $$

$$ V_{n\text{, Shopping}} = 1.25 + 0.1 - 0.2 + 0.15 + 0.3 + 0.25 $$

$$ V_{n\text{, Shopping}} = 1.85 $$

This is the deterministic component of the utility of shopping for this individual. Similarly, utilities for other activities would be calculated.

To get the probability of this individual choosing "Shopping" over other activities, we would then plug this $ V_{n\text{, Shopping}} $ value into the MNL probability formula mentioned above, and normalize it over the sum of exponentiated utilities of all possible activities.

## Options for Variables

The utility functions can be further enriched with interaction terms or non-linear transformations of the attributes (e.g., squared terms, log-transformations). The choice of these terms is often based on domain knowledge, data exploration, and model testing.

Additionally, some variables might be better represented as categorical (e.g., household type) and can be dummy-encoded, leading to multiple columns/variables in the model. In such cases, the interpretation of the coefficients requires care, as the coefficient represents the effect of that category relative to the reference category.

Remember, the primary goal in modeling is not just a good statistical fit but also ensuring the model makes intuitive sense and can be reliably used for predictions or policy analyses.

Certainly! Once we have our choice model set up, generating an activity sequence involves simulating decisions for an individual based on their attributes and the estimated utilities of the choices.

### Activity Sequence Generation:

1. **Initialization**:
    - Begin the sequence with an initial state (e.g., "Home"). This initial state can be hardcoded or chosen based on certain criteria.
    
2. **Calculate Utilities**:
    - For each available activity option, compute the deterministic utility using the estimated coefficients and the attributes of the individual. 
    $$ V_{nj} = \sum_k \beta_{k} x_{knj} $$
    Where:
        - $ V_{nj} $ is the utility of activity $ j $ for individual $ n $.
        - $ \beta_{k} $ is the estimated coefficient for attribute $ k $.
        - $ x_{knj} $ is the value of attribute $ k $ for individual $ n $ and activity $ j $.
    
3. **Compute Probabilities**:
    - For each activity, compute the probability of it being chosen using the multinomial logit formula:
    $$ P_{nj} = \frac{e^{V_{nj}}}{\sum_j e^{V_{nj}}} $$
    Where:
        - $ P_{nj} $ is the probability of choosing activity $ j $ for individual $ n $.
        - The denominator is the sum of exponentiated utilities for all available activity choices.

4. **Random Choice**:
    - Draw a random number from a uniform distribution between 0 and 1.
    - Select the activity whose cumulative probability range includes this random number. For example, if the random number is 0.4, and the cumulative probabilities of activities are [0.2, 0.6, 1.0], the second activity would be chosen.
    
5. **Update Constraints**:
    - After a choice is made, some constraints might need to be applied:
        - Ensure no consecutive trips are the same.
        - Remove any choices that are no longer feasible due to prior decisions or external constraints.
        - Adjust the attributes for the individual if necessary (e.g., if they traveled and are now in a different location).
    
6. **End Condition**:
    - Continue the process until a certain end condition is met. For instance:
        - A fixed number of activities have been generated.
        - The sequence ends with a specific state, like returning home.
    
7. **Post-processing**:
    - After generating the entire sequence, any post-processing can be done, such as ensuring a logical flow or making corrections based on higher-level constraints.

### Worked Out Example:

Let's consider a simple scenario:
- We have 3 activities: Home (1), Work (2), and Shopping (3).
- Individual Attributes: Age=25, Gender=Male, and so on.
- Estimated Coefficients: (from our model).

For the first choice:
- Start at "Home".
- Calculate utilities for Work and Shopping.
- Use the utilities to compute the probabilities.
- Randomly select an activity based on the probabilities.

Suppose "Work" is chosen:
- Update the sequence: Home -> Work.
- Ensure the next choice isn't another "Work".
- Recompute utilities and probabilities for the next choice.

This process continues until the individual returns home or another stopping condition is met.

It's important to note that the generated sequences, while based on estimated parameters, involve a stochastic component and can vary from one simulation to another. The goal is to produce sequences that are representative of real-world behavior.